In [40]:
# Importing Qiskit
from qiskit import *
from qiskit.qasm2 import dumps
from qiskit_aer import Aer

import base64
import numpy as np
# import operation as op
from functools import reduce
import random
from random import randrange

In [2]:
%run ./Hamming.ipynb    # For error correction

# 1. Qubit Manipulation

In [3]:
def NoisyChannel(qc1, qc2, qc1_name, errors, noise = 5e-4):
    ''' This function takes the output of a circuit qc1 (made up only of x and 
        h gates), simulates a noisy quantum channel where Pauli errors (X - bit flip; Z - phase flip)
        will occur in qc2, and then initializes another circuit qc2 with the introduced noise.
    ''' 
    
    # Retrieve quantum state from qasm code of qc1
    qs = [dumps(qc1[i]).split('\n') for i in range(len(qc1))]
    
    # Process the code to get the instructions
    parsed_instructions = []
    for i, qasm_code in enumerate(qs):
        for line in qasm_code:
            line = line.strip()    # removing leading/trailing whitespace
            if line.startswith(('x', 'h', 'measure')):
                line = line.replace('0', str(i))
                parsed_instructions.append(line)
    
    # Apply parsed instructions to qc2
    for instruction in parsed_instructions:
        if instruction.startswith('x'):
            old_qr = int(instruction.split()[1][2:-2])
            qc2[old_qr].x(0)
            
        elif instruction.startswith('h'):
            old_qr = int(instruction.split()[1][2:-2])
            qc2[old_qr].h(0)
        
        elif instruction.startswith('measure'):
            continue    # exclude measuring
            
        else:
            print(f"Unable to parse instruction: {instruction}")
            raise Exception('Unable to parse instruction')
    
    # Introducing noise (taking input)
    for instruction in parsed_instructions:
        if random.random() < noise:
            old_qr = int(instruction.split()[1][2:-2])
            qc2[old_qr].x(0)     # Apply bit-flip error
            errors[0] += 1
            
        if random.random() < noise:
            old_qr = int(instruction.split()[1][2:-2])
            qc2[old_qr].z(0)     # Apply phase-flip error
            errors[1] += 1

    return errors

In [4]:
def generate_random_bits(num):
    """This function generates a random array of bits(0/1) of size = num"""
    bits = np.array([random.randint(0, 1) for _ in range(num)])    # Randomly fills the array with 0/1

    # bit_string = ""
    # for _ in range(num):
    #     rand_bit = random.randint(0, 1)     # Flip Coin
    #     bit_string += str(rand_bit)
        
    return bits

In [5]:
def generate_random_bases(num_of_bases):
    """This function selects a random basis for each bit"""
    
    bases_string = ""
    for _ in range(num_of_bases):
        randBasis = random.randint(0, 1)     # Flip Coin

        if randBasis == 0:
            bases_string += "Z" 
        else:
            bases_string += "X"
            
    return bases_string

In [6]:
def encode(bits, bases):
    """This function encodes each bit into the given basis."""
    
    encoded_qubits = []
    
    for bit, basis in zip(bits, bases):
        qc = QuantumCircuit(1, 1)     # Create a quantum circuit for each qubit
        
        # Possible Cases
        if bit == "1" :
            qc.x(0)

        if basis == 'X' :
            qc.h(0)
            
        encoded_qubits.append(qc)
            
    return encoded_qubits

In [7]:
def measure(qubits, bases):
    """This function measures each qubit in the corresponding basis chosen for it.
        - qubits : a series of 1-qubit Quantum Circuit
        - bases : a string of random [X, Z] bases"""

    bits = np.zeros(len(bases)).astype('uint8')    # The results of measurements

    for idx, (qubit, basis) in enumerate(zip(qubits, bases)):

        if basis == "X" :
            qubit.h(0)
            
        qubit.measure(0, 0)
        
        # Execute on Simulator
        simulator = Aer.get_backend('qasm_simulator')
        transpiled_circuit = transpile(qubit, simulator)
        result = simulator.run(transpiled_circuit, shots=1).result()
        counts = result.get_counts()
        measured_bit = max(counts, key=counts.get)     # Max doesn't matter for simulator since there is only one shot.

        bits[idx] = int(measured_bit)
        
    print(bits)
        
    return bits

In [44]:
def array_to_string(array):
    result = np.array2string(
        array, 
        separator="", 
        max_line_width=(len(array)+3))
    return result.strip('[').strip(']')


In [42]:
def convert_to_octets(key):

    octets = []
    num_octets = len(key) // 8

    for i in range(num_octets):
        start = i * 8
        end = start + 8
        octet = key[start:end]
        octets.append(int(octet, 2))

    return bytearray(octets)

# 2. Preparation

In [8]:
### Use the function Order(bits) to get 'order'.

In [9]:
# Data_length = [1024, 512, 256, 128]
# parity_bits = np.array([np.log2(Data_length[i]) for i in range(len(Data_len))]) + 1    # [10 + 1, 9 + 1, 8 + 1, 7 + 1]
# KEY_LENGTH =  Data_length - parity_bits(Data_length)

eve_presence = True
sl = 8
ch_noise = 2e-4
error_threshold = 0.15

DATA_LENGTH = 2**(11 - sl)
# DATA_LENGTH = 512    # sl = 2
KEY_LENGTH = int(DATA_LENGTH - np.log2(DATA_LENGTH))
Unprocessed_key_len = 2*DATA_LENGTH

# Preparation for encoding
random.seed(0)    # Seed the random number generator. This will be used as our "coin flipper"

# order = np.ceil(np.log2(Unprocessed_key_len)).astype(int)
order = Order(Unprocessed_key_len)
dim = int(2**(order/2))
print(f"{Unprocessed_key_len = }, {KEY_LENGTH = }, {DATA_LENGTH = }, {order = }")

Unprocessed_key_len = 16, KEY_LENGTH = 5, DATA_LENGTH = 8, order = 4


## 2.1 Alice

In [10]:
# Generating a random string of bits
# If KEY_RESERVOIR(ALICE, BOB) exists, then alice_bits = KEY_RESERVOIR[:KEY_LENGTH]. Break the iteration here.

alice_bits = generate_random_bits(Unprocessed_key_len)
alice_bases = generate_random_bases(Unprocessed_key_len) # Alice randomly chooses a basis for each bit.

print(f" Alice uncorrected {block(alice_bits, order)}")

 len(bits) = 16 
 Alice uncorrected block : 
 [[1 1 0 1]
 [1 1 1 1]
 [1 0 0 1]
 [0 0 1 0]] 
 Shape of the block : 4*4


### 2.1.1 Quantum Encoding : 
**Encode the states into quantum circuits**

In [11]:
# Encode Alice's bits
encoded_qubits = encode(alice_bits, alice_bases)

# 3. Quantum Signal Channel
This part can also be simulated by brute forcing 1-2 bit error at random.

## 3.1 Eve
0 : Not present,    1 : present

In [12]:
if eve_presence == 'Random': eve = random.randint(0, 1)
else: eve = int(eve_presence)
    
label = 'Eve' if eve else 'Alice'

In [13]:
qubits_received = [QuantumCircuit(1, 1) for _ in range(len(encoded_qubits))]    # Initializing the circuit
errors_recorded = np.array([0, 0]).astype('uint16')    # Will keep track of the errors INJECTED deliberately by the algorithm

if eve : 
    #print("Eve Present!")
    qubits_intercepted = [QuantumCircuit(1, 1) for _ in range(len(encoded_qubits))]
    
    errors_recorded = NoisyChannel(encoded_qubits, qubits_intercepted, 'Alice', errors_recorded, noise = ch_noise) ##Eve intercepts noisy states     

    eve_bases = generate_random_bases(Unprocessed_key_len) # Generate a random set of bases
    eve_bits = measure(qubits_intercepted, eve_bases) # Measure the qubits
    
    # Eve encodes her decoy qubits and sends them along the quantum channel    
    encoded_intercepted_qubits = encode(eve_bits, eve_bases)    
    errors_recorded = NoisyChannel(encoded_intercepted_qubits, qubits_received, 'Eve', errors_recorded, noise = ch_noise) ## Eve sends noisy states to Bob

else : 
    errors_recorded = NoisyChannel(encoded_qubits, qubits_received, 'Alice', errors_recorded, noise = ch_noise) ## Alice sends noisy states to Bob


[0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0]


## 3.2 Bob

In [14]:
eve_bits

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], dtype=uint8)

In [15]:
bob_bases = generate_random_bases(Unprocessed_key_len) # Bob randomly chooses a basis for each bit.

# Measurement
bob_bits = measure(qubits_received, bob_bases)

print(f"\nAlice sent {block(alice_bits, order)} \n\nBob measured {block(bob_bits, order)}")

[0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0]
 len(bits) = 16 
 len(bits) = 16 

Alice sent block : 
 [[1 1 0 1]
 [1 1 1 1]
 [1 0 0 1]
 [0 0 1 0]] 
 Shape of the block : 4*4 

Bob measured block : 
 [[0 1 0 0]
 [0 0 0 0]
 [0 1 1 0]
 [0 0 0 0]] 
 Shape of the block : 4*4


# 4. Public Interaction Channel

* On completion of this step, the length of bits will cut down to half of the original size.
* Alice can share a string suggesting in which order to use the received bits through QSC. 
* Alice can announce the PARITY_DICT (after sifting)

## 4.1 Sifting

In [16]:
BROADCAST = alice_bases    # Alice tells Bob which bases she used. BROADCAST uses classical channel

In [17]:
# Store the indices of the bases they share in common
common_bases = [i for i in range(Unprocessed_key_len) if bob_bases[i] == BROADCAST[i]]
bob_bits = [bob_bits[index] for index in common_bases]

In [18]:
BROADCAST = common_bases    # Bob tells Alice which bases they shared in common

In [19]:
alice_bits = [alice_bits[index] for index in BROADCAST]    # Alice keeps only the bits they shared in common

In [20]:
alice_bits

[1, 1, 1, 1, 1, 1, 1, 0]

## 4.2 Reconciliation 
(Comparision -- Spotting)
**Key size now reduced to half the original size**

In [21]:
# # What if Eve had listened to the previous round of key share and not this one? The remaining block would then be of mixed errors.
# # Therefore, if Eve is suspected, delete the entire lot of keys

# KEY_RESERVOIR_ALICE = np.concatenate(alice_bits[KEY_LENGTH])
# KEY_RESERVOIR_BOB = np.concatenate(bob_bits[KEY_LENGTH])

# alice_bits = KEY_RESERVOIR_ALICE[:KEY_LENGTH]
# bob_bits = KEY_RESERVOIR_BOB[:KEY_LENGTH]

# KEY_RESERVOIR_ALICE = KEY_RESERVOIR_ALICE[KEY_LENGTH:]
# KEY_RESERVOIR_BOB = KEY_RESERVOIR_BOB[KEY_LENGTH:]

In [22]:
print(len(alice_bits))

8


In [23]:
sample = len(alice_bits)//3    # len(alice_bits) >= 3
errors_detected = 0

for _ in range(sample):
    bit_index = random.randrange(len(alice_bits)) 
    
    if alice_bits[bit_index] != bob_bits[bit_index]:  errors_detected += 1    #calculating errors
        
    del alice_bits[bit_index] #removing tested bits from key strings
    del bob_bits[bit_index]

# order = np.ceil(np.log2(len(alice_bits))).astype(int)
order = Order(alice_bits)

## 4.3 QBER

In [24]:
print(errors_recorded, errors_detected, sample)

[0 0] 1 2


In [25]:
### QBER should be ~ 0.5 (instead of ~0.25) in presence of Eve, because the sample size is 1/3 of the bits AFTER sifting.

QBER = round(errors_detected/sample, 5) # calculating QBER and saving the answer to two decimal places
QBER

0.5

In [26]:
print(f"\n Error Threshold : {error_threshold}")

if QBER > error_threshold:
    if eve : print(" Eve detected : ", end = " ")
    else : print(" Eve FALSELY detected : ", end = " ")
    print("Key not secure")
    # continue    ####### Uncomment this when running this file from iterated_bb84 in a loop
    
else :
    if eve : print('Eve went unnoticed : ', end = " ")
    else : print('Eve not present : ', end = " ")

    # KEY_RESERVOIR = np.concatenate(KEY_RESERVOIR, alice_bits[KEY_LENGTH:]
    print("Key is secure")



 Error Threshold : 0.15
 Eve detected :  Key not secure


# 5. Error Correction

**If the QBER is below a certain threshold, proceed to error correction, else ABORT**

### 5.0 Creating some dummy data for Alice

## 5.1 Encoding parity bits in Alice's key

**alice_bits vs alice_block** : alice_bits contains the secure key (of length : key_len), whereas alice_block contains the redundant fillers(if any) alongwith the parity embedding

In [27]:
key_len = len(alice_bits)
key_len

6

In [28]:
# After Sifting
PARITY_DICT, _ = parity(order)   # Returns empty PARITY_DICT, bin_rep

alice_block = create_parity_block(alice_bits, order, PARITY_DICT)    # Encodes parity on the block
# alice_bits, PARITY_DICT = encode_parity(alice_block, order2, PARITY_DICT)

loc = 7

 Hamming :  2 errors found
 err_count = 2, loc = 7, binary_rep = '111'

 Hamming Results :  (2, 7, '111')
 len(bits) = 8 
 Uncorrected bit string(Order is odd, can't project to a block) : 
 [0 0 0 1 0 1 1 1] 
 Shape of the block : (8,)


In [29]:
BROADCAST = PARITY_DICT

In [30]:
bob_block = create_parity_block(bob_bits, order, BROADCAST)
len(bob_block)
# bob_bits, _ = encode_parity(bob_block, order2)
### Both blocks have been created, now the hamming protocol can be applied

loc = 7

 Hamming :  2 errors found
 err_count = 2, loc = 7, binary_rep = '111'

 Hamming Results :  (2, 7, '111')
 len(bits) = 8 
 Uncorrected bit string(Order is odd, can't project to a block) : 
 [1 1 1 0 1 0 0 0] 
 Shape of the block : (8,)


8

In [31]:
err_count, loc, binary_rep = hamming(bob_block, order)
print(len(bob_block), bob_block)

loc = 7

 Hamming :  2 errors found
 err_count = 2, loc = 7, binary_rep = '111'
8 [1 1 1 0 1 0 0 0]


# Bob

In [32]:
order

3

In [33]:
print(f"Error counts : {err_count} loc : {loc} ({binary_rep})")
total_errors_recorded = errors_recorded[0] + errors_recorded[1]

if err_count != 0 :
    try : 
        if err_count == 1: bob_bits[loc] = np.mod(bob_bits[loc] + 1, 2)

    except :
        raise('KeyError : Location Invalid')

print(f"\nAlice { block(alice_block, order) } \n\nBob { block(bob_block, order) }")

if all(bob_block) == all(alice_block) : 
    print(f"\n Bob bits and alice bits are same")
# print(f'{out[0]:04b}') 
bob_block[:100], alice_block[:100]

Error counts : 2 loc : 7 (111)
 len(bits) = 8 
 len(bits) = 8 

Alice bit string(Order is odd, can't project to a block) : 
 [0 0 0 1 0 1 1 1] 
 Shape of the block : (8,) 

Bob bit string(Order is odd, can't project to a block) : 
 [1 1 1 0 1 0 0 0] 
 Shape of the block : (8,)

 Bob bits and alice bits are same


(array([1, 1, 1, 0, 1, 0, 0, 0], dtype=uint8),
 array([0, 0, 0, 1, 0, 1, 1, 1], dtype=uint8))

In [34]:
QBER = round(total_errors_recorded/sample, 4) # calculating QBER and saving the answer to two decimal places
QBER

0.0

## Final Key : 

In [35]:
if QBER > error_threshold:
    if eve : print(" Eve detected : ", end = " ")
    else : print(" Eve FALSELY not detected : ", end = " ")
    print("Key not secure")

else :
    if eve : print('Eve went unnoticed : ', end = " ")
    else : print('Eve not present : ', end = " ")
    print("Key is secure")


Eve went unnoticed :  Key is secure


In [59]:
key = []
for bit in alice_bits:    # Or bob_bits, since both should be the same
    key.append(bit)

key = np.array(key)

In [67]:
print(key)

[1 1 1 1 1 1]


**Generate alphanumeric keys from the sequences of 0's and 1's**

If you need keys that are made of letters (e.g. for tokens or other regular use), you should group the key into octets,  
create a bytearray of octets and then encode each octet according to ASCII.

Please note that this shortens they key and to work properly quite a key of at least 8 bit is needed.

In [69]:
key_string = np.array2string(key, separator = "").lstrip('[').rstrip(']')
dec = int(key_string, 2)
hexadecimal = hex(dec)
dec, hexadecimal[2:]

(63, '3f')

In [66]:
ASCII_key = base64.b64encode(convert_to_octets(array_to_string(key))).decode('ascii')
print(ASCII_key)

In [50]:
ASCII_key

''

In [ ]:
total_errors_recorded, sample